In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
import tensorflow.keras as keras
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import warnings

warnings.filterwarnings('ignore')

In [ ]:
df  = pd.read_csv('/kaggle/input/insurance-prediction/insurance.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
sns.heatmap(df.corr(),annot=True)

In [ ]:
sns.countplot(df['sex'])

In [ ]:
sns.countplot(df['smoker'])

In [ ]:
smoker_chrg = df[df['smoker'].isin([1])]['charges'].mean()
non_smoker_chrg = df[df['smoker'].isin([0])]['charges'].mean()
print("Mean Smoker charge is {} and non smoker's charge is {}".format(smoker_chrg,non_smoker_chrg))

### This concludes that insurance charge for smoker's are higher than non smokers

In [ ]:

df['age_bins'] = pd.cut(x = df['age'],bins=[17,30,45,64])

In [ ]:
fig,ax = plt.subplots(figsize=(8,5))
sns.countplot(df['age_bins'],ax=ax)

In [ ]:
df.groupby(by='age_bins')['charges'].mean().plot()

### We can clearly see that as the age is increasing, insurance charges are also getting high

In [ ]:
avg_chrg_sex  = df.groupby(by='sex')['charges'].mean()
print('Average charge for female is {} and for male is {}'.format(avg_chrg_sex[0],avg_chrg_sex[1]))

In [ ]:
#Generating new feature based on no. of childrens
df['married'] = np.where(df['children']==0,0,1)


In [ ]:
df.groupby('children')['charges'].mean().plot()
plt.ylabel('charges')

In [ ]:
df['zscore'] = abs(np.round(zscore(a=df['charges']),2))

In [ ]:
df.head()

In [ ]:
sns.distplot(df['charges'])

# Splitting, Scaling & Training

In [ ]:
X = df.drop(columns=['age_bins','charges','zscore'])
y = df['charges']

In [ ]:
trainX,testX,trainY,testY = train_test_split(X,y,random_state=21,test_size=0.20)
scale = StandardScaler()
trainX,testX = scale.fit_transform(trainX),scale.fit_transform(testX)

In [ ]:
def pred_result(testY,pred,model):
    print('--------------- {} -------------'.format(model))
    print('mae: {}'.format(mean_absolute_error(testY,pred)))
    print('mse: {}'.format(mean_squared_error(testY,pred)))
    print('rmse: {}'.format(math.sqrt(mean_squared_error(testY,pred))))
    print('r2_score: {}'.format(r2_score(testY,pred)))

In [ ]:
model = LinearRegression()
model.fit(trainX,trainY)
cv_score = cross_val_score(model,trainX,trainY,cv=10)
pred = model.predict(testX)
pred_result(testY,pred,'LinearRegression')
print('cv_score: {}'.format(cv_score.mean()))

In [ ]:
model = RandomForestRegressor(max_depth=3,random_state=0,n_estimators=45)
cv_score = cross_val_score(model,trainX,trainY,cv=10)
model.fit(trainX,trainY)
pred = model.predict(testX)
pred_result(testY,pred,'RandomForestRegressor')
print('cv_score: {}'.format(cv_score.mean()))